---
title: 3.7 前向传播、反向传播和计算图
date: 2024-2-7 14:00:00
tags: [机器学习,pytorch]
categories: [机器学习]
comment: true
toc: true
---
#
<!--more-->
# 7 前向传播、反向传播和计算图
- 以带dropout的单层隐藏层mlp为例
## 7.1 前向传播
- 按顺序（从输入层到输出层）计算和存储神经网络中每层的结果s。
    - 1）输入是$\mathbf{x} \in \mathbb{R}^d$，隐藏层不含偏置,得到中间变量：
$$\mathbf{z}=\mathbf{W}^{(1)}\mathbf{x}$$
    - 2）激活：
    $$\mathbf{h}=\phi